In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import gmaps

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
# Import cities file as DataFrame
top_counties_pd = pd.read_csv("data/top_counties_Sale_list_Ratio.csv")
top_counties_pd.head()

,Unnamed: 0,RegionName,StateName,Month,Sale to List Ratio
0,20289,San Francisco,CA,2015-05,1.168614
1,69870,Benton,WA,2018-08,1.142615
2,64480,Santa Clara,CA,2018-04,1.134173
3,64545,San Mateo,CA,2018-04,1.126408
4,62838,Box Elder,UT,2018-02,1.114800


In [3]:
# Add columns for lat, lng
# Note that we used "" to specify initial entry.
top_counties_pd["Lat"] = ""
top_counties_pd["Lng"] = ""

top_counties_pd.head()

,Unnamed: 0,RegionName,StateName,Month,Sale to List Ratio,Lat,Lng
0,20289,San Francisco,CA,2015-05,1.168614,,
1,69870,Benton,WA,2018-08,1.142615,,
2,64480,Santa Clara,CA,2018-04,1.134173,,
3,64545,San Mateo,CA,2018-04,1.126408,,
4,62838,Box Elder,UT,2018-02,1.114800,,


In [4]:

params = {"key": gkey}

# Loop through the counties and run a lat/long search for each
for index, row in top_counties_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    county = row['RegionName']
    state = row['StateName']

    # update address key value
    params['address'] = f"{county},{state}"

    # make request
    counties_lat_lng = requests.get(base_url, params=params)
    
    # convert to json
    counties_lat_lng = counties_lat_lng.json()

    top_counties_pd.loc[index, "Lat"] = counties_lat_lng["results"][0]["geometry"]["location"]["lat"]
    top_counties_pd.loc[index, "Lng"] = counties_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
top_counties_pd.head()

,Unnamed: 0,RegionName,StateName,Month,Sale to List Ratio,Lat,Lng
0,20289,San Francisco,CA,2015-05,1.168614,37.7749,-122.419
1,69870,Benton,WA,2018-08,1.142615,46.3166,-119.502
2,64480,Santa Clara,CA,2018-04,1.134173,37.3541,-121.955
3,64545,San Mateo,CA,2018-04,1.126408,37.563,-122.326
4,62838,Box Elder,UT,2018-02,1.114800,41.538,-113.192


In [5]:
# Save Data to csv
top_counties_pd.to_csv("output/county_output_top.csv")



In [6]:
# Create counties dataframe
top_counties_df = pd.read_csv('output/county_output_top.csv')
top_counties_df.dropna()
top_counties_df.head()

,Unnamed: 0,Unnamed: 0.1,RegionName,StateName,Month,Sale to List Ratio,Lat,Lng
0,0,20289,San Francisco,CA,2015-05,1.168614,37.774929,-122.419415
1,1,69870,Benton,WA,2018-08,1.142615,46.316636,-119.502158
2,2,64480,Santa Clara,CA,2018-04,1.134173,37.354108,-121.955236
3,3,64545,San Mateo,CA,2018-04,1.126408,37.562992,-122.325525
4,4,62838,Box Elder,UT,2018-02,1.114800,41.538009,-113.191802


In [7]:
# Store latitude and longitude in locations
locations = top_counties_df[["Lat", "Lng"]]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,  
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:

# Plost as a Hybrid map
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))